<a href="https://colab.research.google.com/github/omakaroshka/Makarevich-OV/blob/main/VKR_Makarevich_3ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# импорт библиотек

In [ ]:
# импорт библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import scipy.stats as stats
import sklearn.linear_model as lm
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler, OrdinalEncoder

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
import pickle


In [ ]:
# Подключаю доступ к файлам данных для гугл драйв
from google.colab import drive
drive.mount('/content/drive/')


In [ ]:
# Пути для гугл драйв
nb_path = '/content/drive/MyDrive/Colab Notebooks/'

models_path = '/content/drive/MyDrive/Colab Notebooks/models/'

In [ ]:
# Пути для локального запуска
nb_path = ''

models_path = '../models/'

# ⦁	Разработать приложение с графическим интерфейсом 

In [ ]:
!gdown --id  1ee_9iNqrYziOGMaV94z2jhWmkOHYq9Wt
df= pd.read_excel('makarevich.xlsx', index_col=0)

In [ ]:
from flask import Flask, request, render_template

import tensorflow as tf

app = Flask(__name__)

def prediction(params):
    model = tf.keras.models.load_model('models/mn_model_nn')
    pred = model.predict([params])
    return pred

@app.route('/', methods=['POST', 'GET'])
def predict():
    message = ''
    if request.method == 'POST':
        param_list = ('Плотность, кг/м3', 'модуль упругости, ГПа', 'Количество отвердителя, м.%', 
                      'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2', 'Поверхностная плотность, г/м2	', 
                      'Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Потребление смолы, г/м2',
                      'Угол нашивки, град', 'Шаг нашивки', 'Плотность нашивки')
        params = []
        for i in param_list:
            param = request.form.get(i)
            params.append(param)
        params = [float(i.replace(',', '.')) for i in params]

        message = f'Соотношение матрица-наполнитель: {prediction(params)}'
    return render_template('mn.html', message=message)

if __name__ == '__main__':
    app.run()

In [ ]:
# Признаки для Модуль упругости при растяжении
y1_columns = ['Модуль упругости при растяжении, ГПа']
X1_columns = [
  'Соотношение матрица-наполнитель', 
  'Плотность, кг/м3',
  'модуль упругости, ГПа', 
  'Количество отвердителя, м.%',
  'Содержание эпоксидных групп,%_2', 
  'Температура вспышки, С_2',
  'Поверхностная плотность, г/м2', 
  'Потребление смолы, г/м2',
  'Угол нашивки', 
  'Шаг нашивки', 
  'Плотность нашивки'
]

In [ ]:
y1 = df.loc[:, y1_columns]
X1 = df.loc[:, X1_columns]

In [ ]:
# Разделю переменные на категориальные и численные
X_categorical = ['Угол нашивки']
X1_continuous = [column for column in X1_columns if column not in X_categorical]

In [ ]:
# Препроцессор для задачи Модуль упругости при растяжении
preprocessor1 = ColumnTransformer(transformers=[
        ("continuous", MinMaxScaler(), X1_continuous),
        ("categorical", OrdinalEncoder(), X_categorical)
])

In [ ]:
def save_object_with_pickle(obj, obj_name):
  filename = models_path + obj_name
  file = open(filename, 'wb')
  pickle.dump(obj, file)
  file.close()

In [ ]:
def load_object_with_pickle(obj_name):
  filename = models_path + obj_name
  file = open(filename, 'rb')
  obj = pickle.load(file)
  file.close()
  return obj

In [ ]:
save_object_with_pickle(preprocessor1, 'preprocessor1')

In [ ]:
save_object_with_pickle(best_model1, 'model1_best')